In [1]:
import logging
from models import *
from train_lstm import *
from ax.service.managed_loop import optimize
import pickle


class ParamSearch:

    def __init__(self, x_train, y_train, x_validate, y_validate, epochs, batch_size=100):
        self.x_train = x_train
        self.y_train = y_train
        self.x_validate = x_validate
        self.y_validate = y_validate
        self.epochs = epochs

    def _train_eval(self, params: Dict[str, Union[str, bool, float, int, None]]) -> float:
        lstm = AutoRegressiveLSTM(6, 100, 3)
        lstm.cuda()
        train_silent(lstm, self.x_train, self.y_train, x_validate, y_validate, epochs=self.epochs, batch_size=100, **params)
        return evaluate(lstm, self.x_validate, self.y_validate)

    def run(self):
        best_parameters, values, experiment, model = optimize(
            parameters=[
                {"name": "lr", "type": "range", "bounds": [1e-5, 0.1], "log_scale": True},
                # {"name": "beta1", "type": "range", "bounds": [0, 1]},
                # {"name": "beta2", "type": "range", "bounds": [0.99, 1], "log_scale": True},
                {"name": "lr_factor", "type": "range", "bounds": [0.1, 0.5]},
                {"name": "patience", "type": "range", "bounds": [100, 1500]},
            ],
            evaluation_function=self._train_eval,
            objective_name='mse',
        )
        print("Found best parameters: %s", best_parameters)
        return best_parameters, values, experiment, model

In [ ]:
from src.train_lstm import *
from ax.plot.contour import plot_contour
from ax.utils.notebook.plotting import render, init_notebook_plotting

warmup = 10
x, y = load_dataset('data/simulations/2021-09-18_time_series.npz', warmup=10)
x_train, x_validate, x_test, y_train, y_validate, y_test = split_dataset_deterministic(x, y, 0.7, 0.2, 0.1)
x_train, x_validate, y_train, y_validate = to_torch(x_train, x_validate, y_train, y_validate)

s = ParamSearch(x_train, y_train, x_validate, y_validate, 3_000)
best_params, values, experiment, model = s.run()

print('=== BEST PARAMS ===')
print(best_params)

print('=== VALUES ===')
print(values)


[INFO 09-23 17:32:01] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lr. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-23 17:32:01] ax.service.utils.instantiation: Inferred value type of ParameterType.FLOAT for parameter lr_factor. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-23 17:32:01] ax.service.utils.instantiation: Inferred value type of ParameterType.INT for parameter patience. If that is not the expected value type, you can explicity specify 'value_type' ('int', 'float', 'bool' or 'str') in parameter dict.
[INFO 09-23 17:32:01] ax.modelbridge.dispatch_utils: Using Bayesian Optimization generation strategy: GenerationStrategy(name='Sobol+GPEI', steps=[Sobol for 5 trials, GPEI for subsequent trials]). Iterations after 5 will take longer to generate due

In [ ]:
render(plot_contour(model=model, param_x='lr', param_y='lr_factor', metric_name='mse'))
